In [ ]:
%pip install gooddata-pandas

In [ ]:
from gooddata_pandas import GoodPandas

host = "YOUR HOST"
token = "GD_TOKEN"

gp = GoodPandas(host, token)


metric_name = "Stock_Price"
workspace_id = "GD_WORKSPACE"


frames = gp.data_frames(workspace_id)

df = frames.for_visualization("GD_VISUALIZATION")


In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import mlflow
import mlflow.pyfunc

def train_arima_model(data, order=(5,1,0)):
    model = ARIMA(data, order=order)
    model_fit = model.fit()
    return model_fit

arima_model = train_arima_model(df[metric_name])

forecast_steps = 5 # Forecast next 5 months
forecast = arima_model.forecast(steps=forecast_steps)
print(forecast)

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
import mlflow
import mlflow.pyfunc
import pandas as pd

class ArimaModelWrapper(mlflow.pyfunc.PythonModel):
    def __init__(self, model):
        self.model = model

    def predict(self, context, model_input):
        return self.model.forecast(steps=len(model_input))

def train_arima_model(data, order=(5,1,0)):
    model = ARIMA(data, order=order)
    model_fit = model.fit()
    return model_fit

arima_model = train_arima_model(df[metric_name])

forecast_steps = 5
forecast = arima_model.forecast(steps=forecast_steps)
print(forecast)

wrapped_model = ArimaModelWrapper(model=arima_model)

# Log the model with MLflow
with mlflow.start_run() as run:
    mlflow.pyfunc.log_model(
        artifact_path="arima_model",
        python_model=wrapped_model
    )

# Load the model for prediction
logged_model = f"runs:/{run.info.run_id}/arima_model"
loaded_model = mlflow.pyfunc.load_model(logged_model)

# Make predictions with the loaded model
model_input = pd.DataFrame([0]*forecast_steps)  # Dummy input for the forecast steps
predictions = loaded_model.predict(model_input)
print(predictions)


In [ ]:
CATALOG_NAME = "main"
SCHEMA_NAME = "default"

In [ ]:
run_id = mlflow.search_runs(filter_string='tags.mlflow.runName = "kindly-owl-169"').iloc[0].run_id

In [ ]:
# Register the model to Unity Catalog. 
model_name = f"ARIMA_PREDICTIONS_STOCK_PRICE"
model_version = mlflow.register_model(f"runs:/{run_id}/arima_model", model_name)

import time
# Registering the model takes a few seconds, so add a small delay
time.sleep(15)